## Objective
- connect a customized component to a demo bot

## steps
- present the resources repository
- set up a demo bot ( + story)
- interface with the bot with python
- select a component
- connect a component
- customize a component
- setting up the competition web server + chat window
- questions

### connecting to dialogflow

In [62]:
!pip install dialogflow

In [63]:
import json
import dialogflow_v2beta1 as dialogflow

session_client = dialogflow.SessionsClient.from_service_account_json(
    "dialogflow_serviceaccount.json"
)

with open("dialogflow_serviceaccount.json", "r") as f:
    sacc = json.load(f)
project_id = sacc["project_id"]

In [64]:

def detect_intent(session_id, text, language_code="en"):
    """Returns the result of detect intent with texts as inputs.
    Using the same `session_id` between requests allows continuation
    of the conversation."""

    session = session_client.session_path(project_id, session_id)

    text_input = dialogflow.types.TextInput(text=text, language_code=language_code)

    query_input = dialogflow.types.QueryInput(text=text_input)

    response = session_client.detect_intent(session=session, query_input=query_input)

    return response


In [68]:
detect_intent("xsfsfdgdfgddf24", "savings?")

response_id: "b088134a-7890-482e-a842-f4af0561c0c5-9cc28bb4"
query_result {
  query_text: "savings?"
  action: "account.balance.check"
  parameters {
    fields {
      key: "account"
      value {
        string_value: "savings account"
      }
    }
  }
  all_required_params_present: true
  fulfillment_text: "Here\'s your latest balance:"
  fulfillment_messages {
    text {
      text: "Here\'s your latest balance:"
    }
  }
  output_contexts {
    name: "projects/banking-qvegra/agent/sessions/xsfsfdgdfgddf24/contexts/balance"
    lifespan_count: 5
    parameters {
      fields {
        key: "account"
        value {
          string_value: "savings account"
        }
      }
      fields {
        key: "account.original"
        value {
          string_value: "savings"
        }
      }
    }
  }
  intent {
    name: "projects/banking-qvegra/agent/intents/e43a1f20-41d4-4e79-ad87-8d88221daedc"
    display_name: "account.balance.check"
  }
  intent_detection_confidence: 1.0
  langu

In [76]:
from enum import Enum
class Comps(Enum):
    MAIN = "main"
    REGISTER = "register_vp3"

In [77]:
import uuid
session_id = str(uuid.uuid4())
comp = Comps.MAIN

def respond(uinput):
    global comp
    if comp == Comps.MAIN:
        df_resp = detect_intent(session_id, uinput)
        if df_resp.query_result.intent.display_name == "account.open":
            comp = Comps.REGISTER
            return respond(uinput)
        return df_resp.query_result.fulfillment_text
    elif comp == Comps.REGISTER:
        coco_resp = coco.exchange(comp.value, session_id, uinput)
        if coco_resp.component_done:
            comp = Comps.MAIN
        return coco_resp.response
            
    return "fallback"
    

In [75]:
respond("thanks")

"Anytime. That's what I'm here for."

In [32]:
import coco

In [13]:
sess = coco.ComponentSession("namer_vp3")

In [14]:
sess("hi")

{
 "response": "Will you please tell me your name?",
 "component_done": false,
 "component_failed": false,
 "updated_context": {},
 "confidence": 1,
 "idontknow": false,
 "action_name": null,
 "response_time": 0.0065541267
}

In [33]:
coco.exchange?

Signature:
coco.exchange(
    component_id: str,
    session_id: str,
    user_input: str = None,
    **kwargs,
) -> coco.coco.CoCoResponse
Docstring:
calls coco and try to maintain similar api.
available optional kwargs are:
    user_input
    context

full api spec available at app.coco.imperson.com

Arguments:
    component_id {str} -- the component id from coco app
    session_id {str} -- a randomly generated session id to identify the session

Returns:
    CoCoResponse instance
File:      ~/.pyenv/versions/3.7.2/envs/webinar/lib/python3.7/site-packages/coco/coco.py
Type:      function
